In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [2]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import json

dataset = load_dataset('multi_eurlex', language='en',label_level='level_2',trust_remote_code=True)
dataset2=load_dataset('harvard-lil/cold-french-law')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/47.8k [00:00<?, ?B/s]

multi_eurlex.py:   0%|          | 0.00/138k [00:00<?, ?B/s]

multi_eurlex.tar.gz:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

README.md:   0%|          | 0.00/6.56k [00:00<?, ?B/s]

cold-french-law.csv:   0%|          | 0.00/2.34G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/841761 [00:00<?, ? examples/s]

In [3]:
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset,random_split

# Load the pre-trained EuroVoc classifier
model_name = "FacebookAI/xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=127)
device=torch.device('cuda')
model.load_state_dict(torch.load('drive/MyDrive/best_model_frozen.pt'))
model=model.to(device)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:

dataset = load_dataset('multi_eurlex', language='en',label_level='level_2',trust_remote_code=True)

In [5]:
df21=pd.DataFrame(dataset2['train'])
selected=df21[(df21.texte_nature=='LOI')&(df21.article_etat=='VIGUEUR')]
selected=selected[(selected.article_contenu_text.str.len()<800)&(selected.article_contenu_text.str.len()>300)]
tests=selected.sample(frac=1)[:20]
tests=tests[['texte_titre','texte_contexte','article_contenu_text']]

In [6]:

def classify_document(text, top_k=5):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs.to(device))
        logits = outputs.logits
        probabilities = torch.sigmoid(logits)[0]
        top_probs, top_labels = torch.topk(probabilities, top_k)
        return top_labels.tolist(), top_probs.tolist()

In [7]:

labels=np.array(list(range(127)))
dico=json.load(open('drive/MyDrive/eurovoc_descriptors.json'))
classlabel=dataset['train'].features['labels'].feature
meanings=[dico[lab]['en'] for lab in classlabel.int2str(labels)]

In [8]:
dico={'titre':[], 'contexte':[], 'texte':[], 'predictions':[]}
for text_titre,text_contexte,article_contenu_text in zip(tests['texte_titre'],tests['texte_contexte'],tests['article_contenu_text']):
  top_labels,_=classify_document(article_contenu_text)
  dico['titre'].append(text_titre)
  dico['contexte'].append(text_contexte)
  dico['texte'].append(article_contenu_text)
  dico['predictions'].append(', '.join([meanings[l] for l in top_labels]))

In [9]:
rslt=pd.DataFrame(dico)

In [10]:
rslt

,titre,contexte,texte,predictions
0,Loi n°49-958 du 18 juillet 1949 RENFORCEMENT D...,None,Le ministre de l'économie et des finances est ...,"EU institutions and European civil service, bu..."
1,LOI n° 2016-1087 du 8 août 2016 pour la reconq...,Titre III : AGENCE FRANÇAISE POUR LA BIODIVERS...,Les agents contractuels de droit public des ét...,"European Union law, environmental policy, trad..."
2,Loi n°93-1313 du 20 décembre 1993 QUINQUENNALE...,"Titre IV : Coordination, simplification et éva...","Une loi ultérieure complétera et, au besoin, a...","European Union law, employment, Europe, econom..."
3,LOI organique n° 2011-333 du 29 mars 2011 rel...,TITRE IER : DISPOSITIONS GENERALES\n,Le Défenseur des droits est nommé par décret e...,"EU institutions and European civil service, Eu..."
4,LOI n° 2019-1479 du 28 décembre 2019 de financ...,PREMIÈRE PARTIE : CONDITIONS GÉNÉRALES DE L'ÉQ...,I.-A modifié les dispositions suivantes :\n-Co...,"taxation, European Union law, oil industry, en..."
5,Loi n°46-2339 du 24 octobre 1946 CONTENTIEUX D...,Titre IV : Procédure et voies de recours\nChap...,"En dehors du cas prévu à l'article précédent, ...","EU institutions and European civil service, Eu..."
6,LOI n° 96-542 du 19 juin 1996 relative au cont...,TITRE Ier bis : Dispositions applicables aux s...,Au cours de la période mentionnée au III de l'...,"European Union law, competition, trade, intern..."
7,LOI n° 2008-757 du 1er août 2008 relative à la...,TITRE II : DISPOSITIONS D'ADAPTATION AU DROIT ...,I. ― Dans les conditions prévues par l'article...,"European Union law, environmental policy, dete..."
8,Loi n° 52-1322 du 15 décembre 1952 instituant ...,Titre III : Du contrat de travail\nChapitre II...,"[ L'ordonnance 82-1114 du 23 décembre 1982, a...","employment, political geography, labour market..."
9,LOI n° 2015-177 du 16 février 2015 relative à ...,Titre VII : DISPOSITIONS RELATIVES AUX PROCÉDU...,Dans les conditions prévues à l'article 38 de ...,"EU institutions and European civil service, ex..."


In [11]:
rslt.to_csv('national_law.csv')